<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/01-Preprocesamiento_de_features_correo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de las features adicionales a la Consulta


1. Traigo los datos y los cargo en el dataframe:

In [547]:
import os.path
from os import path
import pandas as pd

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists("01-Correos_Seleccionados_y_Etiquetados.csv")):
  !wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/TFI/data/Correos_Seleccionados_y_Etiquetados.csv

df = pd.read_csv('Correos_Seleccionados_y_Etiquetados.csv', delimiter="|", index_col=False)

df['Clase'].value_counts()


--2020-12-27 19:43:55--  https://raw.githubusercontent.com/jumafernandez/UNLP/master/TFI/data/Correos_Seleccionados_y_Etiquetados.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 448336 (438K) [text/plain]
Saving to: ‘Correos_Seleccionados_y_Etiquetados.csv.1’

Correos_Seleccionad 100%[===================>] 437.83K  --.-KB/s    in 0.03s   

2020-12-27 19:43:56 (12.3 MB/s) - ‘Correos_Seleccionados_y_Etiquetados.csv.1’ saved [448336/448336]



Boleto Universitario                   240
Ingreso a la Universidad               232
Requisitos de Ingreso                  129
Pedido de Certificados                  69
Inscripción a Cursadas                  64
Problemas con la Clave                  51
Reincorporación                         47
Cursadas                                23
Exámenes                                19
Carga de Notas                          17
Consulta por Legajo                     16
Cambio de Carrera                       14
Consulta por Equivalencias              14
Situación Académica                     12
Cambio de Comisión                      11
Datos Personales                        10
Consulta sobre Título Universitario     10
Vacunas Enfermería                      10
Certificados Web                         6
Simultaneidad de Carreras                6
Name: Clase, dtype: int64

2. Verifico los datos y borro columnas no pre-procesables:

In [548]:
df.drop(columns=['Apellido y Nombre', 'Respuesta'], inplace=True)
df.head()

,Fecha,Hora,Legajo,Documento,Carrera,Teléfono,E-mail,Consulta,Clase
0,08-05-2019,10:49:26,169336,33829069,licenciatura en enfermeria(52),1121550750,rolandflorencia@gmail.com,"hola quiero anotarme a las materias ,para el s...",Vacunas Enfermería
1,08-08-2017,12:29:59,150786,33220121,licenciatura en enfermeria(52),1131066251,vane_male@outlook.com,hola buenos días! quería saber cuando voy a po...,Vacunas Enfermería
2,05-31-2017,01:30:49,156535,43455018,contador publico(54),NaN,solangekarg8@gmail.com,hola quisiera saber si en la consulta de situa...,Situación Académica
3,02-05-2018,22:58:24,155395,38859638,licenciatura en trabajo social(5),1566431259,luztopa@hotmail.com,buenas noches. en mi situacion academica apare...,Situación Académica
4,08-06-2016,13:16:16,115623,35756071,contador publico(54),44556937,yanet868@hotmail.com,"hola, quisiera obtener mi promedio o saber co...",Situación Académica


3. Preproceso la fecha (día de la semana, semana del mes, mes y cuatrimestre) de la consulta:

In [549]:
def convierte_fecha(fecha_consulta):
  from datetime import datetime
  # Convierto a fecha
  fecha = datetime.strptime(fecha_consulta, '%m-%d-%Y') 
  return fecha

def dia_semana(fecha):
  # Tomo el día de la semana
  dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
  # dia_semana = dias_semana[fecha.weekday()]
  dia_semana = fecha.weekday()
  return dia_semana

def semana_del_mes(fecha):
  # Tomo la semana del mes
  semana_mes = (fecha.day-1)//7+1
  return semana_mes

def mes(fecha):
  # Tomo el mes
  meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
  # mes = meses[fecha.month-1]
  mes = fecha.month
  return mes

def cuatrimestre(fecha):
  # Defino el cuatrimestre
  cuatrimestre = 1
  if (fecha.month>7):
    cuatrimestre = 2
  return cuatrimestre

def anio(fecha):
    anio = fecha.year
    return anio

# convierte_fecha(df.loc[0, 'Fecha'])
fecha = df['Fecha'].apply(convierte_fecha)
df['dia_semana'] = fecha.apply(dia_semana)
df['semana_del_mes'] = fecha.apply(semana_del_mes)
df['mes'] = fecha.apply(mes)
df['cuatrimestre'] = fecha.apply(cuatrimestre)
df['anio'] = fecha.apply(anio)

In [550]:
df.drop(columns=['Fecha'], inplace=True)
df.head()

,Hora,Legajo,Documento,Carrera,Teléfono,E-mail,Consulta,Clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio
0,10:49:26,169336,33829069,licenciatura en enfermeria(52),1121550750,rolandflorencia@gmail.com,"hola quiero anotarme a las materias ,para el s...",Vacunas Enfermería,0,1,8,2,2019
1,12:29:59,150786,33220121,licenciatura en enfermeria(52),1131066251,vane_male@outlook.com,hola buenos días! quería saber cuando voy a po...,Vacunas Enfermería,1,2,8,2,2017
2,01:30:49,156535,43455018,contador publico(54),NaN,solangekarg8@gmail.com,hola quisiera saber si en la consulta de situa...,Situación Académica,2,5,5,1,2017
3,22:58:24,155395,38859638,licenciatura en trabajo social(5),1566431259,luztopa@hotmail.com,buenas noches. en mi situacion academica apare...,Situación Académica,0,1,2,1,2018
4,13:16:16,115623,35756071,contador publico(54),44556937,yanet868@hotmail.com,"hola, quisiera obtener mi promedio o saber co...",Situación Académica,5,1,8,2,2016


4. Preproceso la hora de la consulta de la siguiente forma:
    - 0-Mañana (6-12 hs),
    - 1-Media-Tarde (12-16 hs)
    - 2-Tarde (16-20 hs)
    - 3-Noche (20-00 hs)
    - 4-Madrugada (00-6 hs)

In [551]:
def convierte_horario(hora_consulta):
  from datetime import datetime
  # Convierto a hora
  hora = datetime.strptime(hora_consulta, '%H:%M:%S').time()
  return hora

def discretiza_horario(horario):
  if (horario.hour>=6 and horario.hour<12):
    rango_horario=0
  elif (horario.hour>=12 and horario.hour<16):
    rango_horario=1
  elif (horario.hour>=16 and horario.hour<20):
    rango_horario=2
  else:
    rango_horario=3 
  return rango_horario

#discretiza_horario(convierte_horario(df.loc[0, 'Hora']))
horario = df['Hora'].apply(convierte_horario)
df['hora_discretizada'] = horario.apply(discretiza_horario)

In [552]:
df.drop(columns=['Hora'], inplace=True)
df.head()

,Legajo,Documento,Carrera,Teléfono,E-mail,Consulta,Clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada
0,169336,33829069,licenciatura en enfermeria(52),1121550750,rolandflorencia@gmail.com,"hola quiero anotarme a las materias ,para el s...",Vacunas Enfermería,0,1,8,2,2019,0
1,150786,33220121,licenciatura en enfermeria(52),1131066251,vane_male@outlook.com,hola buenos días! quería saber cuando voy a po...,Vacunas Enfermería,1,2,8,2,2017,1
2,156535,43455018,contador publico(54),NaN,solangekarg8@gmail.com,hola quisiera saber si en la consulta de situa...,Situación Académica,2,5,5,1,2017,3
3,155395,38859638,licenciatura en trabajo social(5),1566431259,luztopa@hotmail.com,buenas noches. en mi situacion academica apare...,Situación Académica,0,1,2,1,2018,3
4,115623,35756071,contador publico(54),44556937,yanet868@hotmail.com,"hola, quisiera obtener mi promedio o saber co...",Situación Académica,5,1,8,2,2016,1


5. Preproceso el legajo y el DNI:<br />
  Los separo en cuantiles asumiendo que los DNI mas bajos son personas mas grandes y los legajos mas grandes son estudiantes mas nuevos.

In [553]:
import numpy as np
 
def discretiza_atributo(atributo, intervalos=4):
  # Se discretiza según la cantidad de q definidos
  discretizado = pd.qcut(atributo, q=intervalos)
  
  # Nos quedamos con los diferentes intervalos, los ordenamos min-max
  lista_intervalos = discretizado.unique()

  lista_intervalos.sort_values(inplace=True)
  #  Lo casteamos a str
  lista_intervalos = lista_intervalos.astype(str)
  #  Casteamos el atributo a str
  discretizado = discretizado.astype(str)
  # Borro los nan porque los modifico a mano
  tiene_nan = False
  if ('nan' in lista_intervalos):
    tiene_nan = True
    np.delete(lista_intervalos, np.where(lista_intervalos == 'nan'))

  for i in range(intervalos):
    # Si existiera, quiero dejar el nan en el intervalo con etiqueta 0, por eso sumo 1 a i
    discretizado = discretizado.replace(lista_intervalos[i], i+1)
    discretizado = discretizado.replace('nan', 0)

  return discretizado

# Cambio a numericos los Documentos y los discretizo
df["Documento"] = pd.to_numeric(df["Documento"], errors='coerce')
df["dni_discretizado"] = discretiza_atributo(df["Documento"], 8)

# Cambio a numericos los Legajos y los discretizo
df["Legajo"] = pd.to_numeric(df["Legajo"], errors='coerce')
df["legajo_discretizado"] = discretiza_atributo(df["Legajo"], 4)

In [554]:
df["dni_discretizado"].value_counts()

8    125
7    125
6    125
5    125
4    125
3    125
2    125
1    125
Name: dni_discretizado, dtype: int64

In [555]:
df["legajo_discretizado"].value_counts()

0    310
4    173
1    173
3    172
2    172
Name: legajo_discretizado, dtype: int64

6. Genero una variable dummy con la existencia o no de Legajo y Teléfono:

In [556]:
# Tiene legajo?
def posee_valor(atributo):
  return 1-atributo.isna()
  
df['posee_legajo'] = posee_valor(df['Legajo'])

In [557]:
df.drop(columns=['Legajo', 'Documento'], inplace=True)
df.head()

,Carrera,Teléfono,E-mail,Consulta,Clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo
0,licenciatura en enfermeria(52),1121550750,rolandflorencia@gmail.com,"hola quiero anotarme a las materias ,para el s...",Vacunas Enfermería,0,1,8,2,2019,0,3,4,1
1,licenciatura en enfermeria(52),1131066251,vane_male@outlook.com,hola buenos días! quería saber cuando voy a po...,Vacunas Enfermería,1,2,8,2,2017,1,3,3,1
2,contador publico(54),NaN,solangekarg8@gmail.com,hola quisiera saber si en la consulta de situa...,Situación Académica,2,5,5,1,2017,3,8,3,1
3,licenciatura en trabajo social(5),1566431259,luztopa@hotmail.com,buenas noches. en mi situacion academica apare...,Situación Académica,0,1,2,1,2018,3,5,3,1
4,contador publico(54),44556937,yanet868@hotmail.com,"hola, quisiera obtener mi promedio o saber co...",Situación Académica,5,1,8,2,2016,1,3,1,1


In [558]:
# Tiene teléfono? 
df['posee_telefono'] = posee_valor(df['Teléfono'])

In [559]:
df.drop(columns=['Teléfono'], inplace=True)
df.head()

,Carrera,E-mail,Consulta,Clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono
0,licenciatura en enfermeria(52),rolandflorencia@gmail.com,"hola quiero anotarme a las materias ,para el s...",Vacunas Enfermería,0,1,8,2,2019,0,3,4,1,1
1,licenciatura en enfermeria(52),vane_male@outlook.com,hola buenos días! quería saber cuando voy a po...,Vacunas Enfermería,1,2,8,2,2017,1,3,3,1,1
2,contador publico(54),solangekarg8@gmail.com,hola quisiera saber si en la consulta de situa...,Situación Académica,2,5,5,1,2017,3,8,3,1,0
3,licenciatura en trabajo social(5),luztopa@hotmail.com,buenas noches. en mi situacion academica apare...,Situación Académica,0,1,2,1,2018,3,5,3,1,1
4,contador publico(54),yanet868@hotmail.com,"hola, quisiera obtener mi promedio o saber co...",Situación Académica,5,1,8,2,2016,1,3,1,1,1


7. Se extrae el código de Carrera:

In [560]:
def extrae_codigo_carrera(texto_carrera):
  codigo = 0
  if "sin carrera" not in texto_carrera:
    texto_carrera = texto_carrera.split('(')
    texto_carrera = texto_carrera[len(texto_carrera)-1].split(')')
    codigo = int(texto_carrera[0])
  return codigo

# extrae_codigo_carrera(df.loc[0, 'Carrera'])
df['carrera_valor'] = df['Carrera'].apply(extrae_codigo_carrera)

In [561]:
df.drop(columns=['Carrera'], inplace=True)
df.head()

,E-mail,Consulta,Clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor
0,rolandflorencia@gmail.com,"hola quiero anotarme a las materias ,para el s...",Vacunas Enfermería,0,1,8,2,2019,0,3,4,1,1,52
1,vane_male@outlook.com,hola buenos días! quería saber cuando voy a po...,Vacunas Enfermería,1,2,8,2,2017,1,3,3,1,1,52
2,solangekarg8@gmail.com,hola quisiera saber si en la consulta de situa...,Situación Académica,2,5,5,1,2017,3,8,3,1,0,54
3,luztopa@hotmail.com,buenas noches. en mi situacion academica apare...,Situación Académica,0,1,2,1,2018,3,5,3,1,1,5
4,yanet868@hotmail.com,"hola, quisiera obtener mi promedio o saber co...",Situación Académica,5,1,8,2,2016,1,3,1,1,1,54


8. Se verifica el proveedor de correo electrónico:

In [562]:
def servicio_email(consulta, proveedor):  
  if consulta.lower().find(proveedor)==-1:
    return 0
  else:
    return 1
# Correo Gmail?
# df['correo_gmail'] = df['E-mail'].apply(servicio_email, proveedor='gmail')
# Correo yahoo?
# df['correo_yahoo'] = df['E-mail'].apply(servicio_email, proveedor='yahoo')
# Correo hotmail?
# df['correo_hotmail'] = df['E-mail'].apply(servicio_email, proveedor='hotmail')

def extrae_proveedor_correo(texto_correo):
    texto_correo = texto_correo.split('@')
    texto_correo = texto_correo[len(texto_correo)-1].split('.')
    proveedor_correo = texto_correo[0]
    return proveedor_correo

#extrae_proveedor_correo(df.loc[0, 'E-mail'])
df['proveedor_correo'] = df['E-mail'].apply(extrae_proveedor_correo)

In [563]:
df['proveedor_correo'].value_counts()

gmail              442
hotmail            417
yahoo               49
outlook             41
live                31
icloud               4
msn                  2
baires-sa            1
grupoberro           1
hotmai               1
outlool              1
correoargentino      1
gmil                 1
ampinformatica       1
me                   1
facebook             1
gmal                 1
luve                 1
autloock             1
gmai                 1
uno                  1
Name: proveedor_correo, dtype: int64

In [564]:
df.drop(columns=['E-mail'], inplace=True)
df.head()

,Consulta,Clase,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo
0,"hola quiero anotarme a las materias ,para el s...",Vacunas Enfermería,0,1,8,2,2019,0,3,4,1,1,52,gmail
1,hola buenos días! quería saber cuando voy a po...,Vacunas Enfermería,1,2,8,2,2017,1,3,3,1,1,52,outlook
2,hola quisiera saber si en la consulta de situa...,Situación Académica,2,5,5,1,2017,3,8,3,1,0,54,gmail
3,buenas noches. en mi situacion academica apare...,Situación Académica,0,1,2,1,2018,3,5,3,1,1,5,hotmail
4,"hola, quisiera obtener mi promedio o saber co...",Situación Académica,5,1,8,2,2016,1,3,1,1,1,54,hotmail


9. Reordeno la clase para que me quede última:

In [565]:
y = df['Clase']
df.drop(columns=['Clase'], inplace=True)
df.insert(13, "Clase", y, True)

In [566]:
# Tomo una consulta determinada
df.loc[1, 'Consulta']

# Verifico las columnas del Dataframe
df.head()

,Consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,Clase
0,"hola quiero anotarme a las materias ,para el s...",0,1,8,2,2019,0,3,4,1,1,52,gmail,Vacunas Enfermería
1,hola buenos días! quería saber cuando voy a po...,1,2,8,2,2017,1,3,3,1,1,52,outlook,Vacunas Enfermería
2,hola quisiera saber si en la consulta de situa...,2,5,5,1,2017,3,8,3,1,0,54,gmail,Situación Académica
3,buenas noches. en mi situacion academica apare...,0,1,2,1,2018,3,5,3,1,1,5,hotmail,Situación Académica
4,"hola, quisiera obtener mi promedio o saber co...",5,1,8,2,2016,1,3,1,1,1,54,hotmail,Situación Académica


10. Guardo el csv procesado:

In [567]:
from google.colab import drive
drive.mount('drive')

df.to_csv('01-Correos_1er_preprocesamiento.csv', index=False)

!cp 01-Correos_1er_preprocesamiento.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
